# Loading the Data

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,225,226,227,228,229,230,231,232,233,diagnosis
0,-529.701162,174.982487,-35.876838,3.602020,-5.797643,-25.249239,12.047722,25.886184,-1.058898,-2.034783,...,-1.872412,-2.104881,-1.695691,-2.792980,-2.398958,-3.156902,-1.373869,-1.508376,-1.693629,0
1,-599.782432,147.809664,-46.640515,4.144677,-9.928165,0.848449,-0.606978,17.597225,-22.395312,-10.554085,...,-2.785168,-1.281291,-2.611972,-1.346388,-2.034416,-1.935962,-1.724436,-1.999044,-1.841698,0
2,-595.982418,159.975026,-51.043589,-8.425311,-12.933979,-0.356746,1.818584,25.802396,-0.176397,-13.556326,...,-2.284995,-1.763302,-2.989272,-2.327116,-2.589143,-2.539843,-2.147121,-1.738367,-1.715168,0
3,-584.350952,127.562624,-16.837064,17.217644,-5.560631,11.308404,12.438878,22.426435,-19.855075,-0.019009,...,-7.065100,-2.293150,-3.932550,-3.515862,-3.571797,-2.953559,-2.196334,-2.272294,-2.080865,0
4,-587.294808,204.067369,28.259840,-4.473823,-20.880420,18.048471,2.064965,5.962784,-0.678075,-0.234478,...,-3.798594,-3.785754,-2.704524,-1.409541,-1.467655,-1.581570,-1.289716,-1.743360,-1.153081,0


In [4]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,224,225,226,227,228,229,230,231,232,233
0,-604.340832,177.205901,-6.644630,20.191594,-27.908909,-7.207416,5.044891,11.344951,-3.410233,-26.703724,...,-1.981077,-1.352131,-1.353412,-1.374463,-1.546896,-1.444459,-1.295445,-1.156677,-1.522703,-1.208542
1,-496.467964,165.364798,-35.866713,16.879067,-21.520841,-0.522958,-8.930145,29.957344,5.165274,-8.420818,...,-2.815284,-2.714559,-2.220488,-2.624868,-1.874927,-1.519048,-1.921275,-2.036701,-1.397597,-1.695893
2,-537.022138,110.028308,15.857765,8.863791,2.135650,-32.000582,-15.373006,22.384103,18.954845,0.703363,...,-1.993090,-2.906738,-2.039099,-1.568255,-1.479140,-1.716056,-2.360539,-1.679508,-1.624922,-1.461888
3,-569.166876,108.698789,-6.723939,-12.565589,-9.613699,-10.372419,4.019437,24.184162,11.891249,-12.605170,...,-2.284404,-1.721165,-2.118748,-1.583043,-2.206668,-1.575467,-1.621352,-1.446789,-1.817591,-1.638153
4,-561.878181,80.986413,22.007530,5.988105,2.075446,1.407013,10.854182,3.466996,11.391252,0.567970,...,-4.410798,-3.343444,-2.542235,-2.349626,-1.729869,-1.471224,-2.293651,-2.335223,-2.102965,-1.446605


In [9]:
X_tot = train.drop('diagnosis', axis = 1)
y_tot = train['diagnosis']
X_test = test

#  Scaling

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()
scaler.fit(X_tot)
X_tot = scaler.transform(X_tot)

In [14]:
X_test = scaler.transform(X_test)

# Neural Nets

## Building the network

In [15]:
from keras import models
from keras import layers
from keras.utils import to_categorical

C:\Users\kevin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1024, activation='relu', input_shape = (234,)))
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dense(4, activation = 'softmax'))
    
    model.compile(optimizer = 'rmsprop',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
    return model

## Validating our approach using Stratified K-fold validation

In [23]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

Let's do a stratified five-fold cross-validation

In [30]:
def cross_val_score(X, y, epochs, cv=5 ):
    '''
    Return the mean of the accuracy scores of the model
    on the validation set
    '''
    skf = StratifiedKFold(n_splits=cv, random_state = 42)
    all_scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[test_index], y[test_index]
    
    #One hot enconding the target variable
        y_train = to_categorical(y_train)
        y_val = to_categorical(y_val)
    
    #Compiling the model
        model = build_model()
    
    #Train the model in silent mode(verbose = 0)    
        model.fit(X_train,y_train , epochs=epochs, batch_size=1, verbose=0)

#Evaluate the data on the validation data
        loss, acc = model.evaluate(X_val, y_val, verbose=0)
        all_scores.append(acc)
        
    return np.mean(all_scores)

In [29]:
print('Cross validation Accuracy_score: {:.3f}'.format(cross_val_score(X_tot, y_tot, epochs = 50 )))

Cross validation Accuracy_score: 0.665
